In [3]:
#import libraries
import pandas as pd 
import numpy as np 

In [4]:
#load dataset
df = pd.read_csv("spam_ham_dataset.csv")

In [5]:
df.drop("Unnamed: 0", axis=1, inplace=True) # preparing data, remove unwatned fearures
df # this df has thee fearures lable, text and label_nam (0 not spam, 1 spam)

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,ham,Subject: industrial worksheets for august 2000...,0


In [6]:
df.label.value_counts()
#this shows the data is unbalanced, so we'll not be only relying on the accuracy score
#tho we can also apply data balacning techniques like
# 1 - Oversampling
# 2 - Undersamping
# 3 - Smote
# Ensemble with understaindg undersamping, etc.

ham     3672
spam    1499
Name: label, dtype: int64

In [7]:
df.shape # check the data shape

(5171, 3)

In [8]:
df.head(4) # print the first 4 rows

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1


In [9]:
# transform the label into numerical represtation --> 1 = spam, 0 = ham, but in this case its already claasified into 0s and 1s
# df["label"] = df["label"].apply(lambda x : 1 if x == "spam" else 0 )

In [10]:
# train_test_split
from sklearn.model_selection import train_test_split

# select dependend and independent features
x = df["text"]
y = df["label_num"]

# call train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size= 0.2, stratify=y)

In [11]:
# at this point we know the type of x is pandas series so we need to transfrom this into numerical represtation --> bag of words

# import CountVectorizer 
from sklearn.feature_extraction.text import CountVectorizer

# assign CountVectorizer to v, and pass stop_words = "english" as a parameter to remove stop words
v = CountVectorizer(stop_words="english")

# transofrm x_train into numerical resprestation and store it in x_train_cv
x_train_cv = v.fit_transform(x_train.values)

x_train_cv

<4136x44045 sparse matrix of type '<class 'numpy.int64'>'
	with 269818 stored elements in Compressed Sparse Row format>

In [12]:
# now let's train the model
from sklearn.naive_bayes import MultinomialNB

# create a object of the model --> MultinomialNB
model = MultinomialNB()

# train the model
model.fit(x_train_cv, y_train)

MultinomialNB()

In [13]:
# Model performance evaluation
from sklearn.metrics import classification_report

# trasnform x_test into numerical represtation --> vectors
x_test_cv = v.transform(x_test)

# predit x_test_cv and store it int y_pred
y_pred = model.predict(x_test_cv)

# display the classification report
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       735
           1       0.97      0.96      0.96       300

    accuracy                           0.98      1035
   macro avg       0.97      0.97      0.97      1035
weighted avg       0.98      0.98      0.98      1035



# After text cleaning
## Now let's clean the text and retry all the above steps to see the impact of clearning the text

In [34]:
# import libraries

import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

# cleaning_Text --> lemmatization, tokenization and removing puctuatiuon

def clean_text(text):
    # Remove punctuation and lowercase
    text = ''.join([c for c in text if c not in string.punctuation]).lower()

    # Tokenize and remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in nltk.word_tokenize(text) if token not in stop_words]

    # Lemmatize tokens
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens] 

[nltk_data] Downloading package stopwords to C:\Users\Zeeshan
[nltk_data]     Afridi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Zeeshan
[nltk_data]     Afridi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Zeeshan
[nltk_data]     Afridi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
# Create the new column using vectorized operations for efficiency
df["lemmatized_text"] = df["text"].apply(clean_text)

In [36]:
# Print the DataFrame to verify
print(df["lemmatized_text"])

0       [subject, enron, methanol, meter, 988291, foll...
1       [subject, hpl, nom, january, 9, 2001, see, att...
2       [subject, neon, retreat, ho, ho, ho, around, w...
3       [subject, photoshop, window, office, cheap, ma...
4       [subject, indian, spring, deal, book, teco, pv...
                              ...                        
5166    [subject, put, 10, ft, transport, volume, decr...
5167    [subject, 3, 4, 2000, following, noms, hpl, ta...
5168    [subject, calpine, daily, gas, nomination, jul...
5169    [subject, industrial, worksheet, august, 2000,...
5170    [subject, important, online, banking, alert, d...
Name: lemmatized_text, Length: 5171, dtype: object


In [37]:
df.shape

(5171, 4)

In [38]:
df.head(3)

,label,text,label_num,lemmatized_text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0,"[subject, enron, methanol, meter, 988291, foll..."
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,"[subject, hpl, nom, january, 9, 2001, see, att..."
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,"[subject, neon, retreat, ho, ho, ho, around, w..."


In [39]:
#import train_test_spilt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

c_x = df["lemmatized_text"]
c_y = df["label_num"]

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.2, stratify=y)

In [40]:
cv = CountVectorizer(stop_words= "english")

# transofrm x_train into numerical resprestation and store it in x_train_cv
x_train_cv = cv.fit_transform(x_train.values)

In [41]:
from sklearn.naive_bayes import MultinomialNB

c_model = MultinomialNB()

c_model.fit(x_train_cv, y_train)

MultinomialNB()

In [42]:
from sklearn.metrics import classification_report

x_test_cv =  cv.transform(x_test)

y_pred = c_model.predict(x_test_cv)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       735
           1       0.96      0.96      0.96       300

    accuracy                           0.97      1035
   macro avg       0.97      0.97      0.97      1035
weighted avg       0.97      0.97      0.97      1035



## Testing model


In [48]:
# test the model with new data
emails = [
     "Meeting Reminder: Project Update Session Today @ 2 PM",
    "Welcome to the Team, Ali! Your onboarding starts Monday.",
    "Congratulations! You've won a free consultation.",
    "John Invitation: Join us for an exciting evening!",
    "Feedback requested: Please share your thoughts on our new product.",
    "Important information regarding your recent order.",
    "Happy Birthday, Zeeshan! We have a special offer for you.",
    "Xeven is hiring! We have exciting opportunities for you.",
    "Reminder: Your subscription payment is due.",
    "We've missed you! Come back and enjoy 20% off.",
    "You won't believe this incredible secret to wealth!",
    "Urgent! Your account is at risk! Click here to verify.",
    "Free sample! Just pay shipping and handling.",
    "Guaranteed weight loss! This miracle drug will change your life!",
    "Earn thousands working from home! No experience required!",
    "Click here to claim your free gift!",
    "Exclusive offer! You're pre-approved for a credit card!",
    "Double your money in just 30 days!",
    "This one weird trick will solve your problem!",
    "You've been selected! Don't miss out on this once-in-a-lifetime opportunity!"

    
]

emails_count = v.transform(emails)
preds_uncleaned =model.predict(emails_count)
preds_uncleaned

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [49]:
# test the model with new data
emails = [
  "Meeting Reminder: Project Update Session Today @ 2 PM",
    "Welcome to the Team, Ali! Your onboarding starts Monday.",
    "Congratulations! You've won a free consultation.",
    "John Invitation: Join us for an exciting evening!",
    "Feedback requested: Please share your thoughts on our new product.",
    "Important information regarding your recent order.",
    "Happy Birthday, Zeeshan! We have a special offer for you.",
    "Xeven is hiring! We have exciting opportunities for you.",
    "Reminder: Your subscription payment is due.",
    "We've missed you! Come back and enjoy 20% off.",
    "You won't believe this incredible secret to wealth!",
    "Urgent! Your account is at risk! Click here to verify.",
    "Free sample! Just pay shipping and handling.",
    "Guaranteed weight loss! This miracle drug will change your life!",
    "Earn thousands working from home! No experience required!",
    "Click here to claim your free gift!",
    "Exclusive offer! You're pre-approved for a credit card!",
    "Double your money in just 30 days!",
    "This one weird trick will solve your problem!",
    "You've been selected! Don't miss out on this once-in-a-lifetime opportunity!" 
]

c_emails_count = cv.transform(emails)
preds_cleaned = c_model.predict(c_emails_count)
preds_cleaned

array([0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [51]:
print("Cleaned Data Prediction: ", preds_cleaned)
print("Uncleaned Data Predictino", preds_uncleaned)

Cleaned Data Prediction:  [0 0 1 0 0 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1]
Uncleaned Data Predictino [0 0 1 0 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1]


# Conclusion
#### Till this point both models are performing well but since the classification report says the cleaned text is perofming better so i'll proceed with it.